In [3]:
# 앞에 몽고db랑 연결하는 코드 작성!!!
# 아래는 chatbot 에 purchaselist와 연결되어있는상태
# db연결하고 아래 코드 실행하면 된다


In [1]:
def purchase_search():
    import pymongo
    import pandas as pd
    conn = pymongo.MongoClient()
    chatbot = conn.chatbot
    purchaselist = chatbot.PurchaseList
    list = pd.DataFrame()
    docs = purchaselist.find({})
    for doc in docs:
        df = pd.DataFrame(docs, columns={'user_id', 'purchase_product', 'payment_method', 'payment_total', 'payment_date'})
    list = list.append(df)
    id = input('아이디를 입력하세요 >> ')
    print('1(총 구매 목록), 2(구매 상품), 3(총 구매 가격), 4(결제 건수), 5(특정 시간에 따른 구매 목록)')
    while True:
        print()
        check = int(input('확인하시려는 항목을 선택하세요 >> '))
        print()
        if check == 1:
            search_list = list[list['user_id']==id]
            print('고객님의 구매목록은 다음과 같습니다')
            search_list = pd.DataFrame(search_list, 
                                             columns=['purchase_product', 'payment_method', 'payment_total', 'payment_date'])
            print()
            print(search_list.to_string(index=False))
        elif check == 2:
            search_product = list[list['user_id']==id]['purchase_product']
            print('고객님이 현재까지 구매하신 상품은 다음과 같습니다.')
            print()
            print(','.join(search_product.to_list()))
        elif check == 3:
            search_tprice = list[list['user_id']==id]['payment_total'].astype(int).sum()
            print('고객님의 총 구매 가격은 ' + str(search_tprice) + "입니다")
        elif check == 4:
            list['payment_total'] = list['payment_total'].astype(int)
            total_count = list[list['user_id']=='admin'][['payment_method', 'payment_total']]
            search_payment_method_credit = total_count[total_count['payment_method'] == '신용카드']['payment_method'].unique()[0]
            search_payment_total_credit = total_count[total_count['payment_method'] == '신용카드']['payment_total'].sum()
            search_payment_method_noaccount = total_count[total_count['payment_method'] == '무통장입금']['payment_method'].unique()[0]
            search_payment_total_noaccount = total_count[total_count['payment_method'] == '무통장입금']['payment_total'].sum()
            print(
                  '조회하신 id에 대한 결제 건수는, ' + search_payment_method_credit + ": " + str(search_payment_total_credit) +"원, " +
                  search_payment_method_noaccount + ": " + str(search_payment_total_noaccount) + "원입니다."
            )
        elif check == 5:
            search_date = input('조회하려는 날짜를 선택하세요 (YYYY-mm-dd 형식으로 입력해주세요) >> ')
            search_date_list = list[(list['user_id']==id) & (list['payment_date'].str.contains(search_date))]
            print('조회하신 날짜에 대한 구매목록은 다음과 같습니다.')
            print()
            print(search_date_list.to_string(index=False))
        else:
            print('챗봇을 종료합니다. 이용해주셔서 감사합니다')
            break

In [2]:
purchase_search()

UnboundLocalError: local variable 'df' referenced before assignment